In [1]:
import tensorflow as tf
from tensorflow.python.keras.utils import HDF5Matrix
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import (Input, Lambda, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, 
                                            Lambda, Activation, BatchNormalization, concatenate, UpSampling2D, 
                                            ZeroPadding2D)


from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix


from matplotlib import pyplot as plt
%matplotlib inline

import numpy as np

```Python
x_train = HDF5Matrix("data.h5", "x_train")
x_valid = HDF5Matrix("data.h5", "x_valid")
```
shapes should be:
* (1355578, 432, 560, 1)
* (420552, 432, 560, 1)

In [3]:
def gen_data(shape=0, name="input"):
    data = np.random.rand(512, 512, 4)
    label = data[:,:,-1]
    
    return tf.constant(data.reshape(1,512,512,4).astype(np.float32)), tf.constant(label.reshape(1,512,512,1).astype(np.float32))

## NOTE:
## Tensor 4D -> Batch,X,Y,Z
## Tesnor max. float32!

d, l = gen_data(0,0)
print(d.shape, l.shape)

(1, 512, 512, 4) (1, 512, 512, 1)


In [4]:
def unet():
    inputs, label = gen_data()
    input_shape = inputs.shape
    #down0a = Conv2D(16, (3, 3), padding='same')(inputs)
    down0a = Conv2D(16, kernel_size=(3, 3), padding='same', input_shape=input_shape)(inputs)
    down0a_pool = MaxPooling2D((2, 2), strides=(2, 2))(down0a)
    print("down0a.shape:",down0a.shape,"\ndwnpool.shap:", down0a_pool.shape)#?!? letztes != Batch?
    #dim0 = Batch
    #dim1,dim2 = X,Y
    #dim3 = Kanaele
    
    up1 = UpSampling2D((3, 3))(down0a)
    print("upsamp.shape:",up1.shape)   #UpSampling ändert dim1, dim2... somit (?,X,Y,?) evtl. Batch auf dim0 ?
    
unet()

down0a.shape: (1, 512, 512, 16) 
dwnpool.shap: (1, 256, 256, 16)
upsamp.shape: (1, 1536, 1536, 16)


In [21]:
def unet2(input_shape, output_length):
    inputs = Input(shape=input_shape, name="input")
    
    # 512
    down0a = Conv2D(16, (3, 3), padding='same')(inputs)
    down0a = BatchNormalization()(down0a)
    down0a = Activation('relu')(down0a)
    down0a = Conv2D(16, (3, 3), padding='same')(down0a)
    down0a = BatchNormalization()(down0a)
    down0a = Activation('relu')(down0a)
    down0a_pool = MaxPooling2D((2, 2), strides=(2, 2))(down0a)
    # 256
    down0 = Conv2D(32, (3, 3), padding='same')(down0a_pool)
    down0 = BatchNormalization()(down0)
    down0 = Activation('relu')(down0)
    down0 = Conv2D(32, (3, 3), padding='same')(down0)
    down0 = BatchNormalization()(down0)
    down0 = Activation('relu')(down0)
    down0_pool = MaxPooling2D((2, 2), strides=(2, 2))(down0)
    # 128
    down1 = Conv2D(64, (3, 3), padding='same')(down0_pool)
    down1 = BatchNormalization()(down1)
    down1 = Activation('relu')(down1)
    down1 = Conv2D(64, (3, 3), padding='same')(down1)
    down1 = BatchNormalization()(down1)
    down1 = Activation('relu')(down1)
    down1_pool = MaxPooling2D((2, 2), strides=(2, 2))(down1)
    # 64
    down2 = Conv2D(128, (3, 3), padding='same')(down1_pool)
    down2 = BatchNormalization()(down2)
    down2 = Activation('relu')(down2)
    down2 = Conv2D(128, (3, 3), padding='same')(down2)
    down2 = BatchNormalization()(down2)
    down2 = Activation('relu')(down2)
    down2_pool = MaxPooling2D((2, 2), strides=(2, 2))(down2)
    # 8
    center = Conv2D(1024, (3, 3), padding='same')(down2_pool)
    center = BatchNormalization()(center)
    center = Activation('relu')(center)
    center = Conv2D(1024, (3, 3), padding='same')(center)
    center = BatchNormalization()(center)
    center = Activation('relu')(center)
    # center
    up2 = UpSampling2D((2, 2))(center)
    up2 = concatenate([down2, up2], axis=3)
    up2 = Conv2D(128, (3, 3), padding='same')(up2)
    up2 = BatchNormalization()(up2)
    up2 = Activation('relu')(up2)
    up2 = Conv2D(128, (3, 3), padding='same')(up2)
    up2 = BatchNormalization()(up2)
    up2 = Activation('relu')(up2)
    up2 = Conv2D(128, (3, 3), padding='same')(up2)
    up2 = BatchNormalization()(up2)
    up2 = Activation('relu')(up2)
    # 64
    up1 = UpSampling2D((2, 2))(up2)
    up1 = concatenate([down1, up1], axis=3)
    up1 = Conv2D(64, (3, 3), padding='same')(up1)
    up1 = BatchNormalization()(up1)
    up1 = Activation('relu')(up1)
    up1 = Conv2D(64, (3, 3), padding='same')(up1)
    up1 = BatchNormalization()(up1)
    up1 = Activation('relu')(up1)
    up1 = Conv2D(64, (3, 3), padding='same')(up1)
    up1 = BatchNormalization()(up1)
    up1 = Activation('relu')(up1)
    # 128
    up0 = UpSampling2D((2, 2))(up1)
    up0 = concatenate([down0, up0], axis=3)
    up0 = Conv2D(32, (3, 3), padding='same')(up0)
    up0 = BatchNormalization()(up0)
    up0 = Activation('relu')(up0)
    up0 = Conv2D(32, (3, 3), padding='same')(up0)
    up0 = BatchNormalization()(up0)
    up0 = Activation('relu')(up0)
    up0 = Conv2D(32, (3, 3), padding='same')(up0)
    up0 = BatchNormalization()(up0)
    up0 = Activation('relu')(up0)
    # 256
    up0a = UpSampling2D((2, 2))(up0)
    up0a = concatenate([down0a, up0a], axis=3)
    up0a = Conv2D(16, (3, 3), padding='same')(up0a)
    up0a = BatchNormalization()(up0a)
    up0a = Activation('relu')(up0a)
    up0a = Conv2D(16, (3, 3), padding='same')(up0a)
    up0a = BatchNormalization()(up0a)
    up0a = Activation('relu')(up0a)
    up0a = Conv2D(16, (3, 3), padding='same')(up0a)
    up0a = BatchNormalization()(up0a)
    up0a = Activation('relu')(up0a)
    # 512
    output = Conv2D(1, (1, 1), activation='relu')(up0a)
    
    model = Model(inputs=inputs, outputs=output)
    model.compile(loss="mean_squared_error", optimizer='adam')
    return model

d = unet2((512,512,4),(512,512,1))


Anschließend:
```Python
output_length = 1
input_length = output_length + 1
input_shape=(432, 560, input_length)

model_1 = unet(input_shape, output_length)
model_1.fit(x_train_1, y_train_1, batch_size = 16, epochs = 25, 
            validation_data=(x_valid_1, y_valid_1))
```

In [22]:
d.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 512, 512, 4)  0                                            
__________________________________________________________________________________________________
conv2d_116 (Conv2D)             (None, 512, 512, 16) 592         input[0][0]                      
__________________________________________________________________________________________________
batch_normalization_111 (BatchN (None, 512, 512, 16) 64          conv2d_116[0][0]                 
__________________________________________________________________________________________________
activation_111 (Activation)     (None, 512, 512, 16) 0           batch_normalization_111[0][0]    
__________________________________________________________________________________________________
conv2d_117

In [ ]:
#ToDo: now learn something!